## TensorFlow

In [ ]:
! pip install tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
x = tf.constant(100)
x

In [ ]:
with tf.Session() as sess:
    y = sess.run(x)
    print(y)

In [ ]:
x = tf.constant(2)
y = tf.constant(3)

In [ ]:
with tf.Session() as sess:
    print(sess.run(x+y))
    print(sess.run(x-y))
    print(sess.run(x*y))
    print(sess.run(x/y))

In [ ]:
x = tf.placeholder(tf.int32)
y = tf.placeholder(tf.int32)

In [ ]:
add = tf.add(x,y)
sub = tf.subtract(x,y)
mul = tf.multiply(x,y)

In [ ]:
d = {x:20,y:30}
with tf.Session() as sess:
    print(sess.run(add,feed_dict=d))
    print(sess.run(sub,feed_dict=d))
    print(sess.run(mul,feed_dict=d))

## MNIST Example With TensorFlow

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [3]:
n_hidden_1 = 256 
n_hidden_2 = 256 
n_input    = 784 
n_classes  = 10 
n_samples  = mnist.train.num_examples

In [4]:
with tf.name_scope('inputs'):
    x = tf.placeholder("float", [None, n_input], name= 'input_x')
    y = tf.placeholder("float", [None, n_classes], name= 'input_y')

In [5]:
with tf.name_scope('input_reshape'):
    image_input = tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input', image_input, 10)

In [6]:
def add_layer(x, input_tensors, output_tensors, layer_name, activation_function = None):  
    with tf.name_scope('Layer'):
        with tf.name_scope('Weights'):
            weight = tf.Variable(tf.random_normal([input_tensors, output_tensors]), name = 'w')
            tf.summary.histogram(name = layer_name + '/Weights', values = weight)
        with tf.name_scope('Bias'):
            bias = tf.Variable(tf.random_normal([output_tensors]), name= 'b')
            tf.summary.histogram(name = layer_name + '/Bias', values = bias)
        with tf.name_scope('Wx_plus_b'):
            formula = tf.add(tf.matmul(x, weight), bias)
        if activation_function is None:
            outputs = formula
        else:
            outputs = activation_function(formula)
        tf.summary.histogram(name = layer_name + '/Outputs', values = outputs)
        return outputs

In [7]:
layer1 = add_layer(x, input_tensors = n_input, output_tensors = n_hidden_1, layer_name='layer1',activation_function = tf.nn.relu)
layer2 = add_layer(layer1, input_tensors = n_hidden_1, output_tensors = n_hidden_2, layer_name='layer2',activation_function = tf.nn.relu)
out_layer = add_layer(layer2, input_tensors = n_hidden_2, output_tensors = n_classes, layer_name='out_layer',activation_function = None)

In [8]:
with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out_layer, labels=y))
    tf.summary.scalar('loss', cost)
    
with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
    
with tf.name_scope('Accuracy'):
    acc = tf.equal(tf.argmax(out_layer, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))
    tf.summary.scalar("accuracy", acc)
    

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [9]:
init = tf.global_variables_initializer()

In [10]:
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)

    ## Merge Summary
    
    writer = tf.summary.FileWriter("tensorboard2/", graph = sess.graph)

    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(n_samples/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, c,result = sess.run([optimizer, cost,merged], feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / total_batch
            ## Adding summary of each step
            writer.add_summary(result,  epoch * total_batch + i)

        print("Epoch: {} cost={}".format(epoch+1,avg_cost))

    print("Training Completed in {} Epochs".format(training_epochs))

Epoch: 1 cost=171.17549170754168
Epoch: 2 cost=62.57909491799095
Epoch: 3 cost=45.786427470987526
Epoch: 4 cost=36.984198188781754
Epoch: 5 cost=31.372316499189925
Epoch: 6 cost=27.20156055537139
Epoch: 7 cost=24.033914830467925
Epoch: 8 cost=21.623024080666664
Epoch: 9 cost=19.641191550764162
Epoch: 10 cost=17.9610248836604
Epoch: 11 cost=16.53904706066304
Epoch: 12 cost=15.330048905069177
Epoch: 13 cost=14.25760362925855
Epoch: 14 cost=13.312449820366778
Epoch: 15 cost=12.43925337263467
Training Completed in 15 Epochs


## KERAS

In [11]:
! pip install keras

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### 讀取MNIST 數據

In [31]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Activation, Dense

In [32]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [33]:
x_train.shape

(60000, 28, 28)

In [34]:
x_test.shape

(10000, 28, 28)

In [35]:
x_train = x_train.reshape(60000, 784)
x_test  = x_test.reshape(10000, 784)

In [36]:
x_train.shape

(60000, 784)

In [37]:
x_test.shape

(10000, 784)

In [38]:
x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')
x_train = x_train / 255
x_test  = x_test / 255

In [39]:
y_test

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [40]:
n_classes  = 10 
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test  = keras.utils.to_categorical(y_test, n_classes)

In [41]:
y_test

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### 設定網路參數

In [43]:
n_hidden_1 = 256 
n_hidden_2 = 256 
n_input    = 784 
n_classes  = 10 

training_epochs = 15
batch_size = 100

In [44]:
model = keras.Sequential()
model.add(Dense(n_hidden_1, activation='relu', input_shape=(n_input,)))
model.add(Dense(n_hidden_2, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))

In [45]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [46]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=training_epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 2s 35us/step - loss: 1.0729 - acc: 0.7393 - val_loss: 0.4940 - val_acc: 0.8732
Epoch 2/15
60000/60000 [==============================] - 2s 32us/step - loss: 0.4286 - acc: 0.8858 - val_loss: 0.3567 - val_acc: 0.9000
Epoch 3/15
60000/60000 [==============================] - 2s 32us/step - loss: 0.3462 - acc: 0.9025 - val_loss: 0.3089 - val_acc: 0.9129
Epoch 4/15
60000/60000 [==============================] - 2s 32us/step - loss: 0.3083 - acc: 0.9131 - val_loss: 0.2805 - val_acc: 0.9204
Epoch 5/15
60000/60000 [==============================] - 2s 32us/step - loss: 0.2830 - acc: 0.9196 - val_loss: 0.2616 - val_acc: 0.9249
Epoch 6/15
60000/60000 [==============================] - 2s 32us/step - loss: 0.2633 - acc: 0.9246 - val_loss: 0.2461 - val_acc: 0.9300
Epoch 7/15
60000/60000 [==============================] - 2s 32us/step - loss: 0.2472 - acc: 0.9297 - val_loss: 0.2338 - val_acc

## 預測金融客戶流失

In [47]:
import pandas as pd
dataset = pd.read_csv('https://raw.githubusercontent.com/ywchiu/fuboni/master/data/Churn_Modelling.csv')

HTTPError: HTTP Error 404: Not Found